# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [19]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

In [20]:
def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-08-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-08-05", "%Y-%m-%d")

#Create a range of dates to iterate 
#Note: Periods here represents the number of days it will create from the start date 
#We also do a +2 since it will only generate up to April 29. We inlcude May 1 
#since we want to get data from the last day which is April 30 to May 1 
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#prepare the parameters needed to call the API
sort_type="score"
sort="desc"
fields=["author","subreddit","created_utc","num_comments","score","title","selftext"]
subreddit = 'theumbrellaacademy'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)

Doing 2020-08-01 to 2020-08-02
=====Done
Doing 2020-08-02 to 2020-08-03
=====Done
Doing 2020-08-03 to 2020-08-04
=====Done
Doing 2020-08-04 to 2020-08-05
=====Done
Doing 2020-08-05 to 2020-08-06
=====Done


In [21]:
#Question 3

flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty lists is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head())
#df.to_csv("umbrella.csv")

,author,created_utc,num_comments,score,selftext,subreddit,title
0,cractor28,1596299552,0,1,[removed],theumbrellaacademy,What are Diego's powers?
1,vibing-asian,1596249959,3,1,How and where can I join klaus’s cult,theumbrellaacademy,I would really like to know
2,itzatrap2,1596279233,1,1,,theumbrellaacademy,About Five near the end
3,milliegair,1596285885,0,1,,theumbrellaacademy,my favourite scene in all of series 2 🥺
4,lily99463,1596296823,3,1,"Cha-Cha, is that your mask?\n\nhttps://preview...",theumbrellaacademy,"Easter Egg (Season 2, Episode 8)"


In [23]:
#Question 4 - How many submissions were you able to gather? 


df = pd.read_csv('umbrella.csv')
len(df.index)

403

In [24]:
#Question 4 - Who has the most submissions? 



df = pd.read_csv('umbrella.csv')

#df['author'].value_counts().head(1)
df['author'].value_counts().idxmax()


'AnnaR3b3kah'

In [25]:
#Question 4 - Which submission has the highest score? 

df = pd.read_csv('umbrella.csv')

#df.sort_values(['num_comments'], ascending=[0])

df.nlargest(1,['score'])

,Unnamed: 0,author,created_utc,num_comments,score,selftext,subreddit,title
0,0,cractor28,1596299552,0,1,[removed],theumbrellaacademy,What are Diego's powers?


In [26]:
#Question 4 - Which submission has the highest number of comments?

df = pd.read_csv('umbrella.csv')

#df.sort_values(['num_comments'], ascending=[0])
df.nlargest(1,['num_comments'])

,Unnamed: 0,author,created_utc,num_comments,score,selftext,subreddit,title
97,97,shadowdra126,1596251712,179,1,"Season 2 Episode 9: ""743""\n\nOriginal Air Date...",theumbrellaacademy,"The Umbrella Academy — 2×09 ""743"" — Episode Di..."


In [27]:
#Question 4 - Which day of the week has the most submissions? 
    

df = pd.read_csv('umbrella.csv')
#df["created_utc"].head(20)

df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
df['created_utc'] = pd.to_datetime(df['created_utc'].dt.strftime('%m/%d/%y'))


#df['created_utc'].value_counts().head()

df['created_utc'].value_counts().idxmax()



Timestamp('2020-08-01 00:00:00')